In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import sys

sys.path.append("../")
from modules import common, chapter_08

**Chapter 01**

In [ ]:
# ------------------------------------------------------------
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)


# ------------------------------------------------------------


i = keras.Input(shape=(28 * 28,))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Dense(units=32, kernel_regularizer=keras.regularizers.L2(l2=0.002), activation=tf.nn.relu)(x)
o = keras.layers.Dense(units=10, activation=tf.nn.softmax)(x)
model = keras.Model(inputs=i, outputs=o)
model.summary()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
)


# ------------------------------------------------------------


history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    epochs=10,
    batch_size=512,
    verbose=False,
)
common.plot(data=[history], labels=["mnist"])
loss, accuracy = model.evaluate(x=x_test, y=y_test, verbose=False)
print("Loss: ", loss)
print("Accuracy: ", accuracy * 100)
y_pred = model.predict(x=x_test, verbose=False)
for ground, predict in zip(y_test[:5], y_pred[:5]):
    print("Ground: ", ground)
    print("Predict: ", predict.argmax())
    print("Confidence: ", predict.max())

**Chapter 02**

In [ ]:
# ------------------------------------------------------------
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
B = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(np.tensordot(A, B))
print(np.dot(A, B))
print(A * B)
# Translation: [x y] + [xt yt]
# Rotation: [x y] * [[cos_theta, -sin_theta], [cos_theta, sin_theta]]
# Affine: mx + b

**Chapter 03**

In [ ]:
# ------------------------------------------------------------
class LCTF:
    def __init__(self):
        self.w = tf.Variable(initial_value=np.random.uniform(size=(2, 1)))
        self.b = tf.Variable(initial_value=np.zeros(shape=(1,)))
        self.n = 0.001

    def fit(self, x, y):
        # Forward Propagation
        with tf.GradientTape() as tape:
            y_pred = tf.matmul(x, self.w) + self.b
            loss = tf.reduce_mean(tf.square(y - y_pred))
        # Back Propagation
        gw, gb = tape.gradient(loss, [self.w, self.b])
        self.w -= self.n * gw
        self.b -= self.n * gb
        return loss

    def evaluate(self, x, y):
        y_pred = tf.matmul(x, self.w) + self.b
        loss = tf.reduce_mean(tf.square(y - y_pred))
        return loss

    def predict(self, x):
        return tf.matmul(x, self.w) + self.b


# ------------------------------------------------------------


class LCKeras:
    def __init__(self):
        self.w = tf.Variable(initial_value=np.random.uniform(size=(2, 1)))
        self.b = tf.Variable(initial_value=np.zeros(shape=(1,)))
        self.n = 0.001
        self.loss_fn = keras.losses.MeanSquaredError()
        self.optimizer_fn = keras.optimizers.RMSprop()

    def fit(self, x, y):
        # Forward Propagation
        with tf.GradientTape() as tape:
            y_pred = tf.matmul(x, self.w) + self.b
            loss = self.loss_fn(y, y_pred)
        # Back Propagation
        gradient = tape.gradient(loss, [self.w, self.b])
        self.optimizer_fn.apply_gradients(zip(gradient, [self.w, self.b]))
        return loss

    def evaluate(self, x, y):
        y_pred = tf.matmul(x, self.w) + self.b
        loss = tf.reduce_mean(tf.square(y - y_pred))
        return loss

    def predict(self, x):
        return tf.matmul(x, self.w) + self.b


# ------------------------------------------------------------


class LCKerasv2:
    def __init__(self):
        i = keras.Input(shape=(2, 1))
        x = keras.layers.Rescaling(1.0 / 255)(i)
        o = keras.layers.Dense(units=1, activation=None)(x)
        self.model = keras.Model(inputs=i, outputs=o)
        self.n = 0.001
        self.loss_fn = keras.losses.MeanSquaredError()
        self.optimizer_fn = keras.optimizers.RMSprop()

    def fit(self, model, x, y):
        history = model.fit(x=x, y=y, validation_split=0.3, verbose=False, batch_size=512, epochs=10)
        return history

    def evaluate(self, model, x, y):
        loss, accuracy = model.evaluate(x=x, y=y, verbose=False)
        return loss, accuracy

    def predict(self, model, x):
        return model.predict(x=x, verbose=False)

**Chapter 04**

- IMDB: Binary Classification
- Reuters: Multi-class Single-label classification
- Boston: Non-linear scalar regression

In [ ]:
def encoding(data, num_words):
    output = np.zeros(shape=(data.shape[0], num_words))
    for index, value in enumerate(data):
        output[index][value] = 1
    return output


def holdout(x, y, validation_split=0.3):
    val_size = int(x.shape[0] * validation_split)
    indices = np.random.permutation(x.shape[0])
    x, y = x[indices], y[indices]
    x_val, y_val = x[:val_size], y[:val_size]
    x_train, y_train = x[val_size:], y[val_size:]
    return x_train, y_train, x_val, y_val

Binary Classification

In [ ]:
# ------------------------------------------------------------
num_words = 10000
imdb = keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)
x_train, x_test = encoding(x_train, num_words), encoding(x_test, num_words)
indices = np.random.permutation(x_train.shape[0])
x_train = x_train[indices]
y_train = y_train[indices]


# ------------------------------------------------------------


i = keras.Input(shape=(num_words,))
x = keras.layers.Dense(units=32, activation=tf.nn.relu)(i)
o = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)
model = keras.Model(inputs=i, outputs=o)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
)


# ------------------------------------------------------------


history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    batch_size=512,
    epochs=10,
    verbose=False,
)
common.plot(data=[history], labels=["imdb"])
loss, accuracy = model.evaluate(x=x_test[:3], y=y_test[:3], verbose=False)
print("Loss: ", loss)
print("Accuracy: ", accuracy)
y_pred = model.predict(x=x_test, verbose=False)
for ground, predict in zip(y_test[:3], y_pred[:3]):
    print("--------------------------------------")
    print("Ground: ", True if ground == 1 else False)
    print("Predict: ", True if predict >= 0.5 else False)

Multi-class Single-label Classification

In [ ]:
# ------------------------------------------------------------
num_words = 10000
reuters = keras.datasets.reuters
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words)
x_train = encoding(x_train, num_words)
x_test = encoding(x_test, num_words)
y_train = np.array(y_train)
y_test = np.array(y_test)
indices = np.random.permutation(x_train.shape[0])
x_train = x_train[indices]
y_train = y_train[indices]


# ------------------------------------------------------------


i = keras.Input(shape=(num_words,))
x = keras.layers.Dense(units=32, activation=tf.nn.relu)(i)
o = keras.layers.Dense(units=46, activation=tf.nn.softmax)(x)
model = keras.Model(inputs=i, outputs=o)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
)


# ------------------------------------------------------------


history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    batch_size=512,
    epochs=10,
    verbose=False,
)
common.plot(data=[history], labels=["imdb"])
loss, accuracy = model.evaluate(x=x_test, y=y_test, verbose=False)
print("Loss: ", loss)
print("Accuracy: ", accuracy)
y_pred = model.predict(x=x_test, verbose=False)
for ground, predict in zip(y_train[:10], y_pred[:10]):
    print("------------------------------------")
    print("Ground: ", ground)
    print("Predict: ", predict.argmax())
    print("Confidence: ", predict.max())

Non-linear Scalar regression

In [ ]:
# ------------------------------------------------------------
boston_housing = keras.datasets.boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()


# ------------------------------------------------------------


mean = x_train.mean()
x_train -= mean
std = x_train.std()
x_train /= std
x_test -= mean
x_test /= std


# ------------------------------------------------------------


i = keras.Input(shape=(13))
x = keras.layers.Dense(units=32, activation=tf.nn.relu)(i)
o = keras.layers.Dense(units=1, activation=None)(x)
model = keras.Model(inputs=i, outputs=o)
model.compile(
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
)


# ------------------------------------------------------------


x_train, y_train, x_val, y_val = holdout(x_train, y_train)
history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_val, y_val),
    batch_size=512,
    epochs=10,
    verbose=False,
)
common.plot(data=[history], labels=["boston"], window_titles=["MSE", "MAE"])
mse, mae = model.evaluate(x=x_test, y=y_test, verbose=False)
print("MSE: ", mse)
print("MAE: ", mae)
y_pred = model.predict(x=x_test, verbose=False)
for ground, predict in zip(y_test[:3], y_pred[:3]):
    print("Ground: ", ground)
    print("Predict: ", predict[0])

**Chapter 05**

In [ ]:
# ------------------------------------------------------------
white_noise = np.random.random(size=(x_train.shape[0], 10))
zero_channel = np.zeros(shape=(x_train.shape[0], 10))
x_train_white = np.concatenate([x_train, white_noise], axis=1)
x_train_zero = np.concatenate([x_train, zero_channel], axis=1)


# ------------------------------------------------------------


i = keras.Input(shape=(13))
x = keras.layers.Dense(units=32, kernel_regularizer=keras.regularizers.L2(l2=0.002), activation=tf.nn.relu)(i)
x = keras.layers.Dropout(rate=0.2)(x)
o = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)

**Chapter 07**

In [ ]:
# ------------------------------------------------------------
num_samples = 1000
num_text = 100
num_tags = 10
num_departments = 10
num_difficulties = 5
num_priority = 1


# ------------------------------------------------------------


I_title = np.random.randint(low=0, high=num_text, size=(num_samples, num_text))
I_text = np.random.randint(low=0, high=num_text, size=(num_samples, num_text))
I_tags = np.random.randint(low=0, high=num_tags, size=(num_samples, num_tags))
O_departments = np.random.randint(low=0, high=num_departments, size=(num_samples, num_departments))
O_difficulties = np.random.randint(low=0, high=num_difficulties, size=(num_samples, num_difficulties))
O_priorities = np.random.random(size=(num_samples, num_priority))


# ------------------------------------------------------------


L_title = keras.Input(shape=(num_text), name="L_title")
L_text = keras.Input(shape=(num_text), name="L_text")
L_tags = keras.Input(shape=(num_tags), name="L_tags")
i = keras.layers.Concatenate()([L_title, L_text, L_tags])
x = keras.layers.Dense(units=32, activation=tf.nn.relu)(i)
L_departments = keras.layers.Dense(units=num_departments, activation=tf.nn.softmax, name="L_departments")(x)
L_difficulties = keras.layers.Dense(units=num_difficulties, activation=tf.nn.softmax, name="L_difficulties")(x)
model = keras.Model(inputs=[L_title, L_text, L_tags], outputs=[L_departments, L_difficulties])
features = model.layers[4].output
L_priorities = keras.layers.Dense(units=1, activation=tf.nn.sigmoid, name="L_priorities")(features)
model = keras.Model(inputs=[L_title, L_text, L_tags], outputs=[L_departments, L_difficulties, L_priorities])
keras.utils.plot_model(model, "../resources/model.png", show_shapes=True, dpi=64)


# ------------------------------------------------------------


model.compile(
    loss={
        "L_departments": keras.losses.CategoricalCrossentropy(),
        "L_difficulties": keras.losses.CategoricalCrossentropy(),
        "L_priorities": keras.losses.MeanSquaredError(),
    },
    metrics={
        "L_departments": [keras.metrics.CategoricalAccuracy()],
        "L_difficulties": [keras.metrics.CategoricalAccuracy()],
        "L_priorities": [keras.metrics.MeanAbsoluteError()],
    },
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
)
model.fit(
    x={"L_title": I_title, "L_text": I_text, "L_tags": I_tags},
    y={"L_departments": O_departments, "L_difficulties": O_difficulties, "L_priorities": O_priorities},
    validation_split=0.3,
    batch_size=512,
    epochs=10,
    verbose=False,
)
evaluation = model.evaluate(
    x={"L_title": I_title, "L_text": I_text, "L_tags": I_tags},
    y={"L_departments": O_departments, "L_difficulties": O_difficulties, "L_priorities": O_priorities},
    verbose=False,
)
y_pred = model.predict(
    x={"L_title": I_title, "L_text": I_text, "L_tags": I_tags},
    verbose=False,
)


# ------------------------------------------------------------


class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        pass

    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_begin(self, batch, logs):
        pass


callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.ModelCheckpoint(monitor="val_categorical_crossentropy", filepath="../resource/model.keras", save_best_only=True),
    keras.callbacks.TensorBoard(log_dir="../logs"),
    CustomCallback(),
]
model.save()
keras.models.load_model()

**Chapter 08**

In [ ]:
def extract_features(conv_base, dataset):
    features, labels = [], []
    for images_batch, labels_batch in dataset:
        images_batch = keras.applications.vgg16.preprocess_input(images_batch)
        features_batch = conv_base.predict(x=images_batch, verbose=False)
        features.append(features_batch)
        labels.append(labels_batch)
    return np.concatenate(features, axis=0), np.concatenate(labels, axis=0)


# ------------------------------------------------------------


augmentation_layer = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomZoom(0.1),
        keras.layers.RandomRotation(0.2),
    ]
)


# ------------------------------------------------------------


i = keras.Input(shape=(28, 28, 1))
x = keras.layers.Rescaling(1 / 255.0)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", strides=1, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.2)(x)
o = keras.layers.Dense(units=10, activation=tf.nn.softmax)(x)
model = keras.Model(inputs=i, outputs=o)


# ------------------------------------------------------------


i = keras.Input(shape=(180, 180, 3))
i = augmentation_layer(i)
x = keras.layers.Rescaling(1 / 255.0)(i)
x = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", strides=1, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.2)(x)
o = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)
model = keras.Model(inputs=i, outputs=o)


# ------------------------------------------------------------


conv_base = keras.applications.vgg16.VGG16(include_top=False, weights="imagenet")
new_base_dir, train_dataset, val_dataset, test_dataset = chapter_08.dataset_batches()
x_train, y_train = extract_features(conv_base, train_dataset)
x_test, y_test = extract_features(conv_base, test_dataset)
x_val, y_val = extract_features(conv_base, val_dataset)
i = keras.Input(shape=(5, 5, 512))
x = keras.layers.Flatten()(i)
x = keras.layers.Dropout(rate=0.2)(x)
o = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)
model = keras.Model(inputs=i, outputs=o)


# ------------------------------------------------------------


conv_base.trainable = False
i = keras.Input(shape=(5, 5, 512))
i = augmentation_layer(i)
x = keras.applications.vgg16.preprocess_input(i)
x = conv_base(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.2)(x)
o = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)
model = keras.Model(inputs=i, outputs=o)


# ------------------------------------------------------------


for layer in conv_base.layers[:-4]:
    layer.trainable = False
i = keras.Input(shape=(5, 5, 512))
i = augmentation_layer(i)
x = keras.applications.vgg16.preprocess_input(i)
x = conv_base(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.2)(x)
o = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)
model = keras.Model(inputs=i, outputs=o)

**Chapter 09**

In [15]:
i = keras.Input(shape=(200, 200, 3))
x = keras.layers.Rescaling(1.0 / 255)(i)
x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=512, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=512, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=512, kernel_size=3, strides=2, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=256, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=256, kernel_size=3, strides=2, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(x)
o = keras.layers.Conv2D(filters=4, kernel_size=3, strides=1, padding="same", activation=tf.nn.softmax)(x)
model = keras.Model(inputs=i, outputs=o)


# ------------------------------------------------------------


i = keras.Input(shape=(200, 200, 3))
x = keras.layers.SeparableConv2D(filters=64, kernel_size=3, strides=2, padding="same", activation=None)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(activation=tf.nn.relu)(x)
r = x
x = keras.layers.SeparableConv2D(filters=256, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
r = keras.layers.SeparableConv2D(filters=256, kernel_size=1, strides=2, activation=None)(r)
x = keras.layers.add([x, r])

---